# Vectara

>[Vectara](https://Vectara.com/docs/) is a API platform for building LLM-powered applications. It provides a simple to use API for document indexing and query that is managed by Vectara and is optimized for performance and accuracy. 


This notebook shows how to use functionality related to the `Vectara` vector database. 

See the [Vectara API documentation ](https://Vectara.com/docs/) for more information on how to use the API.

We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Vectara
from langchain.document_loaders import TextLoader

In [ ]:
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

## Connecting to Vectara from LangChain

The Vectara API provides simple API endpoints for indexing and querying.

TBD: corpus_id, customer_Id

In [ ]:
vectara = Vectara.from_documents(docs)

## Similarity search

The simplest scenario for using Vectara is to perform a similarity search. 

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vectara.similarity_search(query)

In [ ]:
print(found_docs[0].page_content)

## Similarity search with score

Sometimes we might want to perform the search, but also obtain a relevancy score to know how good is a particular result.

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vectara.similarity_search_with_score(query)

In [ ]:
document, score = found_docs[0]
print(document.page_content)
print(f"\nScore: {score}")

## Maximum marginal relevance search (MMR)

If you'd like to look up for some similar documents, but you'd also like to receive diverse results, MMR is method you should consider. Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents.

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vectara.max_marginal_relevance_search(query, k=2, fetch_k=10)

In [ ]:
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

## Vectara as a Retriever

Vectara, as all the other vector stores, is a LangChain Retriever, by using cosine similarity. 

In [ ]:
retriever = vectara.as_retriever()
retriever

It might be also specified to use MMR as a search strategy, instead of similarity.

In [ ]:
retriever = vectara.as_retriever(search_type="mmr")
retriever

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
retriever.get_relevant_documents(query)[0]